# Local registration with ICP

    In the RGBD folder we have the first 400 images from one of the datasets from: (http://redwood-data.org/indoor_lidar_rgbd/download.html)


    If you want to display directly in jupyter notebook replace the **draw_registrations** with this:

```python
from open3d import JVisualizer

def draw_registrations(source, target, transformation = None, recolor = False):
        source_temp = copy.deepcopy(source)
        target_temp = copy.deepcopy(target)
        if(recolor):
            source_temp.paint_uniform_color([1, 0.706, 0])
            target_temp.paint_uniform_color([0, 0.651, 0.929])
        if(transformation is not None):
            source_temp.transform(transformation)
        visualizer = JVisualizer()
        visualizer.add_geometry(source_temp)
        visualizer.add_geometry(target_temp)
        visualizer.show()
```

In [1]:
from open3d.j_visualizer import JVisualizer
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import copy


# Helper function to draw registrations (reccomended)
def draw_registrations(source, target, transformation = None, recolor = False):
        source_temp = copy.deepcopy(source)
        target_temp = copy.deepcopy(target)
        if(recolor):
            source_temp.paint_uniform_color([1, 0.706, 0])
            target_temp.paint_uniform_color([0, 0.651, 0.929])
        if(transformation is not None):
            source_temp.transform(transformation)
        o3d.visualization.draw_geometries([source_temp, target_temp])
        
def draw_registrations1(source, target, transformation = None, recolor = False):
        source_temp = copy.deepcopy(source)
        target_temp = copy.deepcopy(target)
        if(recolor):
            source_temp.paint_uniform_color([1, 0.706, 0])
            target_temp.paint_uniform_color([0, 0.651, 0.929])
        if(transformation is not None):
            source_temp.transform(transformation)
        visualizer = JVisualizer()
        visualizer.add_geometry(source_temp)
        visualizer.add_geometry(target_temp)
        visualizer.show()

## Creating pointclouds from image data
Now we are going to try to create our own pointclouds from rgb and depth images.

In [ ]:
# Read in images. We have images 000000 - 0000400
color_raw0 = o3d.io.read_image("RGBD/color/000000.jpg")
depth_raw0 = o3d.io.read_image("RGBD/depth/000000.png")

color_raw1 = o3d.io.read_image("RGBD/color/000005.jpg")
depth_raw1 = o3d.io.read_image("RGBD/depth/000005.png")

Create pointclouds from rgb + depth images.

If you set *convert_rgb_to_intensity = False* you will retain the colors from the rgb image.

In [ ]:
rgbd_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw0, 
    depth_raw0, 
    convert_rgb_to_intensity = False)

rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw1, 
    depth_raw1, 
    convert_rgb_to_intensity = True)

#show images
fig= plt.figure(figsize=(15,15))
plt.subplot(221)
plt.title('Redwood grayscale0 image')
plt.imshow(rgbd_image0.color)

plt.subplot(222)
plt.title('Redwood depth0 image')
plt.imshow(rgbd_image0.depth)

plt.subplot(223)
plt.title('Redwood grayscale1 image')
plt.imshow(rgbd_image1.color)

plt.subplot(224)
plt.title('Redwood depth1 image')
plt.imshow(rgbd_image1.depth)

plt.show()


## Images to Pointcloud
Now we create point clouds from the rgbd images we just created.


Here we use **PinholeCameraIntrinsicParameters.PrimeSenseDefault** as default camera parameter. 

It has image resolution 640x480, focal length (fx, fy) = (525.0, 525.0), and optical center (cx, cy) = (319.5, 239.5). 

An identity matrix is used as the default extrinsic parameter. pcd.transform applies an up-down flip transformation on the point cloud for better visualization purpose.


If it becomes too slow you can downsample the pointcloud.

In [ ]:
# Source pointcloud
camera = o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

source = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image0, camera)

# Target pointcloud
target = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image1, camera)

# Flip it, otherwise the pointcloud will be upside down
source.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
target.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Draw
draw_registrations(source, target, recolor=True)

### Evaluation of pointclouds

Before we can run ICP we evaluate our source and target pointclouds. This gives us a measure to see if we need a better initial transformation or not.

The function evaluate_registration calculates two main metrics:

    1. fitness: measures the overlapping area (# of inlier correspondences / # of points in target). The higher the better.
    2. inlier_rmse: measures the RMSE of all inlier correspondences. The lower the better.
(http://www.open3d.org/docs/latest/tutorial/Basic/icp_registration.html)

In [ ]:
# Parameters
threshold = 0.03
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

#Evaluate registration
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

## ICP

Now try to call icp with your point clouds and your initial transformation.

Initially we use:
```Python
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

icp_result = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    point_to_plane)

```
   registration_icp is called with a different parameter TransformationEstimationPointToPlane. 
   Internally, this class implements functions to compute the residuals and Jacobian matrices of the point-to-plane ICP objective.

In [17]:
###
# ICP: PointToPlane
###
def ICP_point2plane(source, target, threshold, trans_init):
    
    point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

    source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30), fast_normal_computation = True)
    target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30), fast_normal_computation = True)

    icp_result = o3d.pipelines.registration.registration_icp(source,
                                                             target,
                                                             threshold,
                                                             trans_init,
                                                             point_to_plane)
#     print(icp_result)
#     print("Transformation is:")
#     print(icp_result.transformation)
    return icp_result.transformation

# draw_registrations(source, target, icp_result.transformation, True) # not original color
# draw_registrations(source, target, icp_result.transformation) # original color

In [ ]:
###
# ICP: PointToPoint
###
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint()

source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30), fast_normal_computation = True)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30), fast_normal_computation = True)

icp_result = o3d.pipelines.registration.registration_icp(source,
                                                         target,
                                                         threshold,
                                                         trans_init,
                                                         point_to_point)

print(icp_result)
print("Transformation is:")
print(icp_result.transformation)

draw_registrations(source, target, icp_result.transformation, True) # not original color

# Exersices

### A)
If you increase the amount of steps from the original image so from i.e. 000000-000001 to 00000-000300 what happens?
### B)
Can you tweak the parameters *threshold* and *trans_init* to combat some of the ill effects that starts appearing?
### C)
Again try to use 
```Python
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    point_to_plane)
```

This requires you to find the normals for each point cloud use:
```python
    source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                            max_nn=30),fast_normal_computation=True)
```
Compare the resulting translations of the two methods is one better than the other?
### D)
Extend this and try to see how much of the bedroom you can reconstruct from the rgb and depth images.
you can extend a pointcloud by new = source + target remember to resample the point cloud some times so it does not get too large down_source = source.voxel_down_sample(voxel_size=0.05)

In [21]:
import glob

# Source pointcloud
camera = o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

# Parameters
threshold = 0.03
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

images_color = glob.glob('RGBD/color/*.jpg')
assert images_color

images_depth = glob.glob('RGBD/depth/*.png')
assert images_depth

color_raw0 = o3d.io.read_image(images_color[0])
depth_raw0 = o3d.io.read_image(images_depth[0])

rgbd_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw0, 
    depth_raw0, 
    convert_rgb_to_intensity = False)

source = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image0, camera)
down_source = source.voxel_down_sample(voxel_size=0.05)

for fname in images_color:
    # read the first image        
    
    color_raw = o3d.io.read_image(images_color[1])
    depth_raw = o3d.io.read_image(images_depth[1])

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw, 
        depth_raw, 
        convert_rgb_to_intensity = False)
    
    # Target pointcloud
    target = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera)
    down_target = target.voxel_down_sample(voxel_size = 0.05)
    
    icp_result = ICP_point2plane(down_source, target, threshold, trans_init)
    
    down_source += target
    down_source = source.voxel_down_sample(voxel_size = 0.05)

draw_registrations(down_source, down_target, icp_result.transformation, True)

AttributeError: 'numpy.ndarray' object has no attribute 'transformation'

In [25]:
images_color = glob.glob('RGBD/color/*.jpg')
assert images_color

images_depth = glob.glob('RGBD/depth/*.png')
assert images_depth

color_raw0 = o3d.io.read_image(images_color[0])
depth_raw0 = o3d.io.read_image(images_depth[0])

rgbd_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw0, 
    depth_raw0, 
    convert_rgb_to_intensity = False)
source0 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image0, camera)

color_raw1 = o3d.io.read_image(images_color[55])
depth_raw1 = o3d.io.read_image(images_depth[55])

rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw, 
    depth_raw, 
    convert_rgb_to_intensity = False)
target = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera)

source = source0 + target

draw_registrations(source0, source, recolor = True)